In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 57kB/s 
     |████████████████████████████████| 419kB 35.3MB/s 
     |████████████████████████████████| 61kB 13.2MB/s 
     |████████████████████████████████| 3.0MB 43.9MB/s 


In [0]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [0]:
# import libraries
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from tensorflow.keras import models, layers
from tensorflow import keras

In [0]:
SOURCE_ZIPFILE_ZOOM_1 = 'zoom_1_256_256_partition_test_only.zip'
SOURCE_ZIPFILE_ZOOM_2 = 'zoom_2_256_256_partition_zoom_1_pair_test_only.zip'


In [0]:
BEST_MODEL_WEIGHT = 'combined_vgg_transfer_batch_norm_03-0.9627-0.1151.h5'

In [0]:
zipped_file_url_zoom_1 = 'https://storage.googleapis.com/applied-dl-sj/camelyon/input_data/new/{}'.format(SOURCE_ZIPFILE_ZOOM_1)
zipped_file_url_zoom_2 = 'https://storage.googleapis.com/applied-dl-sj/camelyon/input_data/new/{}'.format(SOURCE_ZIPFILE_ZOOM_2)

In [0]:
!curl -O $zipped_file_url_zoom_1
!curl -O $zipped_file_url_zoom_2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6550M  100 6550M    0     0  56.6M      0  0:01:55  0:01:55 --:--:-- 71.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6512M  100 6512M    0     0  65.2M      0  0:01:39  0:01:39 --:--:-- 88.6M


In [0]:
%%time

# unzip data folders to root dir
zf = zipfile.ZipFile(SOURCE_ZIPFILE_ZOOM_1)
zf.extractall()

zf = zipfile.ZipFile(SOURCE_ZIPFILE_ZOOM_2)
zf.extractall()


CPU times: user 1min 32s, sys: 44.1 s, total: 2min 16s
Wall time: 3min 57s


In [0]:
build_model_code_url = 'https://storage.googleapis.com/applied-dl-sj/camelyon/source_code/model_build.py'

!curl -O $build_model_code_url

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1659  100  1659    0     0   3796      0 --:--:-- --:--:-- --:--:--  3796


In [0]:
model_weight_url = 'https://storage.googleapis.com/applied-dl-sj/camelyon/output_data/best_weights/{}'.format(BEST_MODEL_WEIGHT)

!curl -O $model_weight_url

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.1M  100 89.1M    0     0  44.1M      0  0:00:02  0:00:02 --:--:-- 44.1M


### Preprocess Data

In [0]:
np.random.seed(828)

In [0]:
IMG_DIR_ZOOM_1 = 'zoom_1_256_256_partition_test_only'
IMG_DIR_ZOOM_2 = 'zoom_2_256_256_partition_zoom_1_pair_test_only'

In [0]:
ls zoom_2_256_256_partition_zoom_1_pair_test_only/meta/

partition_meta_info.json


In [0]:
meta_file_name = os.path.join(IMG_DIR_ZOOM_2, 
                              'meta',
                              'partition_meta_info.json')

# get metadata
meta_df = pd.read_json(meta_file_name, dtype=False)

In [0]:
meta_df.shape

(1245104, 8)

In [0]:
# filter
filter_mask = (meta_df['type'] == 'test') & (meta_df['is_non_gray'] == 1) & \
              (meta_df['is_roi'] == 1)


meta_df = meta_df[filter_mask].copy()

In [0]:
meta_df.shape

(19293, 8)

In [0]:
meta_df['label'] = meta_df['label'].astype(str)

In [0]:
meta_df.head()

,file_name,ref_file_name,img_id,is_non_gray,is_roi,label,non_gray_ratio,type
1032041,tumor_slide_002_split_184_58_overlap_128_offse...,tumor_slide_002_split_184_58.png,002,1,1,0,0.939255,test
1032061,tumor_slide_057_split_242_50_overlap_128_offse...,tumor_slide_057_split_242_50.png,057,1,1,0,0.558640,test
1032121,tumor_slide_057_split_249_47_overlap_128_offse...,tumor_slide_057_split_249_47.png,057,1,1,0,0.944595,test
1032175,tumor_slide_002_split_226_123_overlap_128_offs...,tumor_slide_002_split_226_123.png,002,1,1,0,0.688614,test
1032180,tumor_slide_057_split_217_56_overlap_128_offse...,tumor_slide_057_split_217_56.png,057,1,1,0,0.413834,test


In [0]:
BATCH_SIZE = 64
TARGET_IMG_SIZE = (256, 256)

In [0]:
# specify image data generator with augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

def custom_generator_for_pair(generator, df, zoom_1_dir, zoom_2_dir):

    genX1 = generator.flow_from_dataframe(dataframe=df, 
                                          directory=zoom_1_dir,
                                          x_col='ref_file_name', 
                                          y_col='label',
                                          class_mode='binary', 
                                          target_size=TARGET_IMG_SIZE, 
                                          batch_size=BATCH_SIZE,
                                          drop_duplicates=False,
                                          shuffle=False)
    
    genX2 = generator.flow_from_dataframe(dataframe=df, 
                                          directory=zoom_2_dir,
                                          x_col='file_name', 
                                          y_col='label',
                                          class_mode='binary', 
                                          target_size=TARGET_IMG_SIZE, 
                                          batch_size=BATCH_SIZE,
                                          drop_duplicates=False,
                                          shuffle=False)

    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[0]], X2i[1]  #Yield both images and their mutual label

In [0]:
test_generator = custom_generator_for_pair(datagen,
                                           meta_df,
                                           os.path.join(
                                                IMG_DIR_ZOOM_1,
                                                'test',
                                                'slide'),
                                           os.path.join(
                                                IMG_DIR_ZOOM_2,
                                                'test',
                                                'slide'),
                                           
                                           )

In [0]:
steps_per_epoch = meta_df.shape[0] // BATCH_SIZE + 1  # +1 added to ensure generator reaches full cycle - if meta_df is perfectly divisible by BATCH_SIZE, +1 should be removed

In [0]:
# temp = np.array([])

# for i in tqdm(range(steps_per_epoch)):
#     temp_batch = next(test_generator)
#     temp = np.concatenate((temp, temp_batch[1]))

In [0]:
# temp.shape, meta_df.shape

In [0]:
# y_label = meta_df['label'].astype(float).values

In [0]:
# assert np.abs(temp - y_label).max() < 1e-8

### Build model

In [0]:
from model_build import build_vgg16_double_input

In [0]:
model = build_vgg16_double_input(input_shape=(256, 256, 3))

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
# compile model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

# print model architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 8, 8, 512)    14714688    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 32768)        0           vgg16[1][0]                  

In [0]:
steps_per_epoch

302

In [0]:
model.load_weights(BEST_MODEL_WEIGHT)

In [0]:
%%time

y_pred_prob = model.predict_generator(test_generator,
                                      steps=steps_per_epoch)

Found 19293 images belonging to 2 classes.
Found 19293 images belonging to 2 classes.
CPU times: user 5min 5s, sys: 1min 41s, total: 6min 46s
Wall time: 7min 16s


In [0]:
y_pred_prob.shape, meta_df.shape

((19293, 1), (19293, 8))

In [0]:
meta_df['y_pred_prob'] = y_pred_prob.flatten()

In [0]:
meta_df.to_pickle('zoom_1_256_256_test_inference_result_dual_input.pkl')